In [ ]:
! pip install azure-search-documents==11.6.0b12 --quiet
! pip install azure-identity --quiet
! pip install openai --quiet
! pip install aiohttp --quiet
! pip install ipykernel --quiet
! pip install requests --quiet

In [7]:
import os

import dotenv
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

openai_client = AzureOpenAI(
        api_key=os.getenv("aozai_api_key"),  
        api_version=os.getenv("aoai_llm_api_version"),
        azure_endpoint=os.getenv("aoai_resource_uri")
    )

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=os.getenv("aoai_embedding_model_name"), input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
search_creds = AzureKeyCredential(os.getenv("AZURE_SEARCH_API_KEY"))

AZURE_SEARCH_FULL_INDEX = "envision-qa-index"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=search_creds)
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=search_creds)

from azure.search.documents.indexes.models import (
    AzureOpenAIVectorizerParameters,
    AzureOpenAIVectorizer,
    CorsOptions,
    HnswAlgorithmConfiguration,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    VectorSearch,
    VectorSearchProfile
)

index = SearchIndex(
    name=AZURE_SEARCH_FULL_INDEX,
    cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=600), 
    fields=[
                SimpleField(name="id", type=SearchFieldDataType.String, key=True,searchable=True, filterable=True, sortable=True, facetable=True),
                SearchableField(name="question", filterable=True, sortable=True, facetable=True,type=SearchFieldDataType.String, analyzer_name="zh-Hans.microsoft"),
                SearchableField(name="answer", filterable=True, sortable=True, facetable=True,type=SearchFieldDataType.String, analyzer_name="zh-Hans.microsoft"),
                SearchableField(name="applicationType", filterable=True, sortable=True, facetable=True,type=SearchFieldDataType.String, analyzer_name="zh-Hans.microsoft"),
                SearchField(name="question_embedding", 
                            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                            hidden=False, 
                            searchable=True, 
                            filterable=False, 
                            sortable=False, 
                            facetable=False,
                            vector_search_dimensions=1536, 
                            vector_search_profile_name="azureOpenAIHnswProfile"),
                SearchField(name="answer_embedding", 
                            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                            hidden=False, 
                            searchable=True, 
                            filterable=False, 
                            sortable=False, 
                            facetable=False,
                            vector_search_dimensions=1536, 
                            vector_search_profile_name="azureOpenAIHnswProfile")
            ],
    semantic_search=SemanticSearch(
                default_configuration_name="default",
                configurations=[
                    SemanticConfiguration(
                        name="default",
                        prioritized_fields=SemanticPrioritizedFields(
                            title_field=SemanticField(field_name="question"),
                            content_fields=[
                                SemanticField(field_name="answer")
                            ],
                            keywords_fields=[
                                SemanticField(field_name="question")
                            ]
                        ),
                    )
                ]
            ),
    vector_search=VectorSearch(
                algorithms=[HnswAlgorithmConfiguration(name="myHnsw")],
                profiles=[VectorSearchProfile(name="azureOpenAIHnswProfile",algorithm_configuration_name="myHnsw",vectorizer_name="azureOpenAIVectorizer")],
                vectorizers=[
                    AzureOpenAIVectorizer(
                        vectorizer_name="azureOpenAIVectorizer",
                        parameters=AzureOpenAIVectorizerParameters(
                            resource_url=os.getenv("aoai_resource_uri"),
                            deployment_name=os.getenv("aoai_embedding_deployment_name"),
                            model_name=os.getenv("aoai_embedding_model_name"),
                            api_key=os.getenv("aozai_api_key")))
                ]
            )
)

index_client.create_or_update_index(index)


In [2]:
# create a agent 

agent_name = "envision-qa-agent"

from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters

agent = KnowledgeAgent(
    name=agent_name,
    models=[
        KnowledgeAgentAzureOpenAIModel(
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=os.getenv("aoai_resource_uri"),
                deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
                model_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
            )
        )
    ],
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=AZURE_SEARCH_FULL_INDEX,
            default_reranker_threshold=2.5
        )
    ],
)

index_client.create_or_update_agent(agent)
print(f"Knowledge agent '{agent_name}' created or updated successfully")

Knowledge agent 'envision-qa-agent' created or updated successfully


In [3]:
instructions = """
An Q&A agent that can answer questions about the envision company's automation office and some internal knowledge.
Sources have a JSON format with a id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "assistant",
        "content": instructions
    }
]

In [4]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams

agent_client = KnowledgeAgentRetrievalClient(endpoint=AZURE_SEARCH_ENDPOINT, agent_name=agent_name, credential=search_creds)

messages.append({
    "role": "user",
    "content": """
    打印机怎么使用？
    """
})

# client autho error
# retrieval_result = agent_client.knowledge_retrieval.retrieve(
#     retrieval_request=KnowledgeAgentRetrievalRequest(
#         messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg["content"])]) for msg in messages],
#         target_index_params=[KnowledgeAgentIndexParams(index_name=AZURE_SEARCH_FULL_INDEX, reranker_threshold=2.5)]
#     )
# )
messages.append({
    "role": "assistant",
    "content": "[{\"ref_id\":0,\"title\":\"如何在新电脑上添加公司的打印机\",\"terms\":\"如何在新电脑上添加公司的打印机\",\"content\":\"请按照以下步骤操作：1. 打开控制面板，选择设备和打印机；2. 点击添加打印机；3. 输入打印机地址：\\\\p.envision.com_x0008_w-ricoh-print。\"},{\"ref_id\":1,\"title\":\"申请彩色打印权限后如何进行彩色打印\",\"terms\":\"申请彩色打印权限后如何进行彩色打印\",\"content\":\"请添加彩色打印机驱动，具体路径为：\\\\\\\\p.envision.com\\\\Color-Ricoh-Print。\"},{\"ref_id\":2,\"title\":\"咨询如何使用打印机\",\"terms\":\"咨询如何使用打印机\",\"content\":\"将驱动设置为默认打印的，将彩打驱动复制到左下角放大镜进行安装\"},{\"ref_id\":3,\"title\":\"咨询已申请彩色打印权限，为何在打印机是还是无法选择彩色呢\",\"terms\":\"咨询已申请彩色打印权限，为何在打印机是还是无法选择彩色呢\",\"content\":\"发送彩色打印机驱动\\\\\\\\p.envision.com\\\\Color-Ricoh-Print\\n并指导安装，已可以正常使用。\"},{\"ref_id\":4,\"title\":\"如何使用打印机\",\"terms\":\"如何使用打印机\",\"content\":\"需要使用工卡刷打印机，进行身份验证才能打印\"},{\"ref_id\":5,\"title\":\"如何解决打印时遇到的报错无法打印文件的问题\",\"terms\":\"如何解决打印时遇到的报错无法打印文件的问题\",\"content\":\"请重装打印驱动并重启电脑。具体操作为：打开控制面板，右键删除连接为\\\\p.envision.com_x0008_w-ricoh-print的打印机驱动，然后将该链接粘贴到桌面左下角的Windows搜索框中，按回车键进行安装。重启电脑后再尝试打印。如果问题仍然存在，请联系人工服务处理。\"},{\"ref_id\":6,\"title\":\"打印机显示脱机，无法打印，应该如何处理？\",\"terms\":\"打印机显示脱机，无法打印，应该如何处理？\",\"content\":\"请删除打印驱动并重新安装。具体步骤为：在控制面板中找到打印驱动并右键删除，然后在Windows搜索框中输入\\\\p.envision.com_x0008_w-ricoh-print并按回车键，按照提示进行安装。如果仍然无法安装，请重启电脑后再尝试添加打印机。\"},{\"ref_id\":7,\"title\":\"如何解决无法连接彩色打印机的问题\",\"terms\":\"如何解决无法连接彩色打印机的问题\",\"content\":\"请安装彩色打印机的驱动，驱动链接为：\\\\p.envision.com\\\"Color-Ricoh-Print\\\"。确保您已获得彩色打印权限。\"},{\"ref_id\":8,\"title\":\"更换新账号后打印权限问题\",\"terms\":\"更换新账号后打印权限问题\",\"content\":\"告知黑白打印默认有权限，\\\\\\\\p.envision.com\\\\BW-Ricoh-Print\\n复制到电脑左下角Windows搜索框里面，按回车就可以安装驱动，打印时选择此驱动。\"},{\"ref_id\":9,\"title\":\"如何安装打印机驱动程序\",\"terms\":\"如何安装打印机驱动程序\",\"content\":\"请联系人工服务处理\"},{\"ref_id\":10,\"title\":\"如何开通彩印权限并安装彩打打印机驱动？\",\"terms\":\"如何开通彩印权限并安装彩打打印机驱动？\",\"content\":\"请访问w3.envision-energy.com，进入W3-所有应用-IT服务-IT账号和服务管理，提交新建申请。安装驱动时，请在控制面板中选择设备和打印机，点击添加打印机，输入\\\\p.envision.com\\\\, 然后选择Color-Ricoh-Print进行添加。\"},{\"ref_id\":11,\"title\":\"如何下载和使用8楼打印机的驱动\",\"terms\":\"如何下载和使用8楼打印机的驱动\",\"content\":\"请复制以下链接到电脑左下角Windows搜索框里面，按回车即可安装驱动：\\\\p.envision.com\\\\BW-Ricoh-Print。打印时需要选择安装好的驱动进行打印操作。\"},{\"ref_id\":12,\"title\":\"如何解决打印驱动安装失败的问题\",\"terms\":\"如何解决打印驱动安装失败的问题\",\"content\":\"请使用驱动链接 \\\\p.envision.com_x0008_w-ricoh-print 进行安装\"},{\"ref_id\":13,\"title\":\"如何在Mac电脑上安装打印机驱动并解决打印问题\",\"terms\":\"如何在Mac电脑上安装打印机驱动并解决打印问题\",\"content\":\"请重新安装打印机驱动并进行设置，确保可以正常使用。如果问题仍然存在，请联系人工服务处理。\"},{\"ref_id\":14,\"title\":\"电脑彩印怎么打印\",\"terms\":\"电脑彩印怎么打印\",\"content\":\"请安装彩色打印机驱动，驱动链接为：\\\\p.envision.com\\\"Color-Ricoh-Print\\\"。点击电脑左下角的放大镜，将链接复制到搜索栏中，打开并安装即可。安装完成后，您可以选择彩色打印机进行打印，打印机会自动匹配。\"},{\"ref_id\":15,\"title\":\"如何安装海兴储能项目的打印机驱动\",\"terms\":\"如何安装海兴储能项目的打印机驱动\",\"content\":\"请通过控制面板进入设备和打印机，选择添加打印机，然后输入路径 \\\\p.envision.com_x0008_w-ricoh-print 进行添加.\"},{\"ref_id\":16,\"title\":\"如何申请彩色打印及安装驱动\",\"terms\":\"如何申请彩色打印及安装驱动\",\"content\":\"请在'IT账号和服务管理流程'中发起彩色打印申请。权限审批完成后，您可以手动安装彩色打印驱动，路径为：\\\\p.envision.com\\\"Color-Ricoh-Print\\\"。将该路径复制到电脑左下角的Windows搜索框中，按回车即可安装驱动。如果安装后仍有问题，请等待24小时以确保权限生效。\"},{\"ref_id\":17,\"title\":\"如何将我的笔记本电脑连接到新加坡办公室的打印机？\",\"terms\":\"如何将我的笔记本电脑连接到新加坡办公室的打印机？\",\"content\":\"请安装打印机驱动程序，链接为：\\\\p.envision.com\\\\BW-Ricoh-Print。建议您回到办公室后尝试连接，如有问题请再次联系。\"},{\"ref_id\":18,\"title\":\"咨询打印驱动下载使用问题\",\"terms\":\"咨询打印驱动下载使用问题\",\"content\":\"\\\\\\\\p.envision.com\\\\BW-Ricoh-Print复制到电脑左下角Windows搜索框里面，按回车安装驱动，打印时选择装好的驱动。\"},{\"ref_id\":19,\"title\":\"如何安装公司打印机驱动\",\"terms\":\"如何安装公司打印机驱动\",\"content\":\"请复制以下路径到左下角Windows搜索框：\\\\p.envision.com_x0008_w-ricoh-print，然后按回车键安装驱动。\"},{\"ref_id\":20,\"title\":\"咨询安装彩色打印机驱动\",\"terms\":\"咨询安装彩色打印机驱动\",\"content\":\"查询用户具备彩打权限\\n发送\\\\\\\\p.envision.com\\\\Color-Ricoh-Print驱动，以协助用户安装\"},{\"ref_id\":21,\"title\":\"安装黑白打印驱动\",\"terms\":\"安装黑白打印驱动\",\"content\":\"\\\\\\\\p.envision.com\\\\BW-Ricoh-Print\\n复制到电脑左下角Windows搜索框里面，按回车就可以安装驱动\"},{\"ref_id\":22,\"title\":\"公司打印机驱动怎么安装\",\"terms\":\"公司打印机驱动怎么安装\",\"content\":\"\\\\\\\\p.envision.com\\\\BW-Ricoh-Print\\n复制到左下角Windows搜索框，按回车键安装驱动，已解决。\"},{\"ref_id\":23,\"title\":\"连接公司打印机打印文件需要安装驱动吗？\",\"terms\":\"连接公司打印机打印文件需要安装驱动吗？\",\"content\":\"请复制以下驱动链接并在左下角的放大镜中粘贴进行安装： \\\\p.envision.com_x0008_w-ricoh-print\"}]"
})

In [11]:
response = openai_client.chat.completions.create(
    messages=messages,
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    
)

In [13]:
import textwrap
import json

wrapped = textwrap.fill(response.choices[0].message.content, width=100)
print(wrapped)


使用打印机需要遵循以下步骤：  1. **安装打印机驱动程序**：    - 打开电脑左下角的Windows搜索框，将以下链接复制粘贴并按回车：      -
黑白打印机驱动：`\\p.envision.com\BW-Ricoh-Print` [ref_id: 21]        - 彩色打印机驱动：`\\p.envision.com\Color-
Ricoh-Print`（需要彩印权限）[ref_id: 14]    - 驱动安装成功后，设置相应打印机为默认打印机。  2. **刷工卡验证身份**：    -
在打印文档前，请将工卡（员工卡）刷在打印机上的身份验证区域完成认证。[ref_id: 4]  3. **选择正确的打印机驱动**：    -
在打印时确保选择已安装和设置的打印机驱动。此外，根据需求选择黑白或彩色打印。  如果遇到问题，例如无法安装或打印，请尝试重启电脑或重装驱动。[ref_id: 6, 12] If still
unresolved,联系人工服务处理。
